## Deep Classification Network

0. First, the grid established in the Dataset exploration notebook will be tested with the SVM. Then, a DNN is implemented.

1. Import all `modules` we are going to need

In [1]:
import torch
from torch import nn
import torchvision

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

2. Import Input Matrix X. Then, import y

In [3]:
import os
import pandas as pd

In [4]:
os.chdir("/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis")

In [5]:
# the number of row in each data frame
# you can put any value here according to your situation
chunksize = 1000
index=0
# the list that contains all the dataframes
list_of_dataframes = []

for df in pd.read_csv('X_Matrix.csv', chunksize=chunksize):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)
    print('Read the {num} dataframe'.format(num=index))
    index=index+1
# if you want all the dataframes together, here it is
#result = pd.concat(list_of_dataframes)

Read the 0 dataframe
Read the 1 dataframe
Read the 2 dataframe
Read the 3 dataframe
Read the 4 dataframe
Read the 5 dataframe
Read the 6 dataframe
Read the 7 dataframe
Read the 8 dataframe
Read the 9 dataframe
Read the 10 dataframe
Read the 11 dataframe
Read the 12 dataframe
Read the 13 dataframe
Read the 14 dataframe


3. Import y

In [9]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/mat2csv/emotic_pre/')
train = pd.read_csv('train.csv')
train_mscoco = train[train['Folder'] == 'mscoco/images']

In [10]:
import re

In [11]:
y = []
i=0
for rating in train_mscoco['Continuous_Labels']:
    y.append(rating)

In [12]:
y_ = []
for idx, label in enumerate(y):
    s = (re.sub(r"[\[\]]",'',y[idx]))
    for i, x in enumerate(s):
        y_.append(x)
        if x == ',':
            y_.remove(x)
        if x == ' ':
            y_.remove(x)
        try:
            if x == '1' and s[i+1] == '0':
                y_.remove(x)
                y_.append('10')
        except IndexError:
            continue
        



In [13]:
for idx, el in enumerate(y_):
    if el == '10':
        if y_[idx+1] == '0':
            y_.remove(y_[idx+1])



In [14]:
len(y_)

42828

In [15]:
cont_labels = []
for val in y_:
    cont_labels.append(float(val))

In [16]:
img = 0
Arousal = []
Valence = []
Dominance = []
for val in cont_labels:
    if img == 0:
        Arousal.append(val)
        img = img+1
        continue
    if img == 1:
        Valence.append(val)
        img = img+1
        continue 
    if img == 2:
        Dominance.append(val)
        img = 0
        continue
    



In [17]:
import numpy as np

Arousal = np.array(Arousal)



In [18]:
Arousal.shape

(14276,)

In [19]:
y = Arousal

In [21]:
list_of_dataframes[14].shape[0]

276

In [22]:
from sklearn.feature_selection import SelectKBest

In [25]:
y_df = pd.DataFrame(y)

In [26]:
k_list = []
track = 0
i=0
j=1000
for chunk in list_of_dataframes:
    k_list.append(pd.DataFrame(SelectKBest(k=40000).fit_transform(chunk, y_df[i:j])))

    print('Applied k-best to {track} chunks!'.format(track=track))
    track=track+1
    if track == 14:
        i=14000
        j=14000+list_of_dataframes[14].shape[0]
        
    else:
        i=i+1000
        j=j+1000
        

/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 0 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 1 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Applied k-best to 12 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 13 chunks!


/home/jpauli/miniconda3/envs/yolo/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Applied k-best to 14 chunks!


In [28]:
k_list[14]

,0,1,2,3,4,5,6,7,8,9,...,39990,39991,39992,39993,39994,39995,39996,39997,39998,39999
0,14000.0,0.002983,0.002624,0.003486,0.004920,0.005720,0.019834,0.013985,0.013921,0.008999,...,0.047873,0.016421,0.010720,0.007967,0.003424,0.028607,0.006359,0.002972,0.003128,0.001388
1,14001.0,0.062586,0.027364,0.053738,0.048039,0.054604,0.064544,0.063026,0.032068,0.022397,...,0.027307,0.008309,0.013371,0.004423,0.013888,0.025754,0.010644,0.014561,0.026044,0.008892
2,14002.0,0.078794,0.039873,0.074930,0.055050,0.069653,0.093185,0.047934,0.013722,0.027485,...,0.035630,0.004622,0.016758,0.020034,0.004449,0.004084,0.010122,0.005570,0.003506,0.009049
3,14003.0,0.037931,0.021864,0.014515,0.005891,0.012321,0.008541,0.009423,0.003371,0.005977,...,0.007062,0.007914,0.007093,0.010805,0.011705,0.002574,0.006702,0.004432,0.003992,0.018727
4,14004.0,0.007292,0.008967,0.019246,0.010443,0.017788,0.037201,0.025179,0.039869,0.022370,...,0.020466,0.018037,0.025835,0.041098,0.025301,0.010871,0.011995,0.011456,0.017219,0.029210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,14271.0,0.267639,0.042201,0.027928,0.089289,0.064527,0.054865,0.083702,0.023402,0.044274,...,0.062407,0.056790,0.055323,0.050115,0.030401,0.087418,0.009426,0.025549,0.030971,0.055721
272,14272.0,0.032026,0.057695,0.034665,0.059026,0.059890,0.029450,0.046175,0.061146,0.042535,...,0.032671,0.030913,0.035982,0.026066,0.018557,0.006037,0.007422,0.008649,0.016959,0.016321
273,14273.0,0.012289,0.010207,0.007706,0.023971,0.007756,0.013266,0.006782,0.009684,0.007833,...,0.006524,0.001503,0.006025,0.001511,0.000982,0.003358,0.015541,0.001342,0.016504,0.011995
274,14274.0,0.084886,0.109931,0.112647,0.063579,0.031083,0.042788,0.013575,0.101139,0.022196,...,0.053313,0.004429,0.009398,0.011244,0.008214,0.008786,0.007248,0.007781,0.026731,0.009179


Y TO DF?
    

Transform X to numpy array